In [5]:

import heat_exchanger
from constants import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [14]:

# read GA3_previous_desgins.csv

df = pd.read_csv('GA3_previous_designs.csv')
df["year"]


0     2022
1     2022
2     2022
3     2022
4     2022
5     2022
6     2022
7     2022
8     2022
9     2022
10    2022
11    2022
12    2023
13    2023
14    2023
15    2023
16    2023
17    2023
18    2023
19    2023
20    2023
21    2023
Name: year, dtype: int64

In [15]:

HX = heat_exchanger.build_heat_exchanger([2],[2], 0.31, Side.OPPOSITE, Pattern.SQUARE)
HX.set_conditions([20, 60])

